In [ ]:
pip install spacy
python -m spacy download en
nlp = spacy.load('en_core_web_sm')

In [ ]:
# create a streaming DataFrame that reads from the Kafka topic
df = spark.readStream.format('kafka') \
    .option('kafka.bootstrap.servers', 'localhost:9092') \
    .option('subscribe', 'assignment3part1_read') \
    .load() \
    .selectExpr('CAST(value AS STRING)')

words = df.select(explode([entity.text for entity in nlp(df.value).ents])).alias('named_entities')
word_count = words.groupBy('named_entities').agg(count('*').alias('count'))

# create a streaming DataFrame that writes the word count to another Kafka topic
word_count.writeStream.format('kafka') \
    .option('kafka.bootstrap.servers', 'localhost:9092') \
    .option('topic', 'assignment3part1_write') \
    .option('checkpointLocation', '/tmp/checkpoint') \
    .start() \
    .awaitTermination()